In [1]:
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, TimeDistributed
import pickle
# import nltk
import itertools
import collections

from keras.preprocessing import sequence
from keras.models import model_from_json
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.optimizers import Adam

import keras.backend as K

import spacy
nlp = spacy.load('en')
# from keras.utils import preprocessing
# from model.textGenModel import TextGenModel

import pandas as pd

INPUT_DIR = ''

Using TensorFlow backend.
/Library/Anaconda3/anaconda3/envs/env1/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
str2=''
with open(INPUT_DIR + 'movie_data/movie_conversations_temp.txt', 'r') as f:
    corpus_text_2 = f.readlines()
#     print(corpus_text_2[0])
    for c in corpus_text_2:
        str2 = ' '.join([str2, c])

print(len(str2))
str2[:200]

16841888


' Did you change your hair?No.You might wanna think about itI missed you.It says here you exposed yourself to a group of freshmen girls.It was a bratwurst.  I was eating lunch.With the teeth of your zi'

In [3]:
# constant token and params for our models
START_TOKEN = "EOS"
END_TOKEN = "EOS"
UNKNOWN_TOKEN = "UNK"
PADDING_TOKEN = "PAD"

In [4]:
def get_words_mappings(tokenized_sentences, vocabulary_size):
    # Using NLTK
    #frequence = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    #vocab = frequence.most_common(vocabulary_size)

    # Using basic counter
    counter = collections.Counter(itertools.chain(*tokenized_sentences))
    vocab = counter.most_common(vocabulary_size)
    index_to_word = [x[0] for x in vocab]
    # Add padding for index 0
    index_to_word.insert(0, START_TOKEN)
    index_to_word.insert(1, PADDING_TOKEN)
    # Append unknown token (with index = vocabulary size + 1)
    index_to_word.append(UNKNOWN_TOKEN)
    word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])
    
    return index_to_word, word_to_index


In [5]:
vocabulary_size = 50000
sent_max_len = 20

# hidden_size = 512
# hidden_size = 128
# embedding_size = 128
# batch_size = 64
max_sent_len = 20
embedding_size = 32
stateful = False

In [6]:
# corpus_text3 = str2
# corpus_tokens_tmp=[]
# for i in range(0, 16000000, 100000):
#     print(i)
#     tmp_corpus = [token.orth_ for token in nlp(corpus_text3[i:i + 100000])]
#     corpus_tokens_tmp = corpus_tokens_tmp + tmp_corpus
# print('Example tokenized excerpt: {}'.format(corpus_tokens_tmp[10:20]))

In [7]:
import pickle
with open ('corpus_token_tmp.dat', 'rb') as p:
    corpus_tokens_tmp = pickle.load(p)

In [8]:
print(len(corpus_tokens_tmp))
vocabulary_size = 50000
index_to_word, word_to_index = get_words_mappings(
                                        [corpus_tokens_tmp], #cause a list of sentences is expected
                                        vocabulary_size)
vocabulary_size = len(index_to_word)
print("Vocabulary size = " + str(vocabulary_size))

3825494
Vocabulary size = 50003


In [9]:
df_word_to_index = pd.DataFrame.from_dict(word_to_index, orient='index')
index_to_word_dict = dict([(i,w) for w,i in word_to_index.items()])
df_index_to_word = pd.DataFrame.from_dict(index_to_word_dict, orient='index')

In [10]:
print(df_word_to_index.head())
print(df_index_to_word.head())

                   0
Brigham        24356
detour?Sump'n  44129
OB             24355
Olympico?No    44130
yes.'So        44141
     0
0  EOS
1  PAD
2    .
3    ,
4    I


In [11]:
df_word_to_index.to_csv('w2i.csv', encoding='utf-8')
df_index_to_word.to_csv('i2w.csv', encoding='utf-8')

In [12]:
df_word_to_index = pd.read_csv('w2i.csv', encoding='utf-8', index_col =0)
df_index_to_word = pd.read_csv('i2w.csv', encoding='utf-8', index_col =0)

In [13]:
word_to_index_dict = df_word_to_index.to_dict(orient='index')
index_to_word_dict = df_index_to_word.to_dict(orient='index')

In [14]:
print({k: word_to_index_dict[k] for k in list(word_to_index_dict)[:5]})
print({k: index_to_word_dict[k] for k in list(index_to_word_dict)[:5]})

{'Brigham': {'0': 24356}, "detour?Sump'n": {'0': 44129}, 'OB': {'0': 24355}, 'Olympico?No': {'0': 44130}, "yes.'So": {'0': 44141}}
{0: {'0': 'EOS'}, 1: {'0': 'PAD'}, 2: {'0': '.'}, 3: {'0': ','}, 4: {'0': 'I'}}


### Inference :

In [15]:
autoencoder_model_loaded = load_model('/Users/ashwins/gitlab_repo/chatbot/full_model_r_new_epoch_3.h5')
autoencoder_model_loaded.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dec_input (InputLayer)          (None, 20)           0                                            
__________________________________________________________________________________________________
enc_input (InputLayer)          (None, 20)           0                                            
__________________________________________________________________________________________________
shared_emb (Embedding)          (None, 20, 32)       1600096     enc_input[0][0]                  
                                                                 dec_input[0][0]                  
__________________________________________________________________________________________________
enc_lstm (LSTM)                 [(None, 32), (None,  8320        shared_emb[0][0]                 
__________

In [16]:
for l in autoencoder_model_loaded.layers:
    print(l)

In [17]:
# encoder_model = Model(encoder_inputs, encoder_states)
encoder_model = load_model('/Users/ashwins/gitlab_repo/chatbot/enc_model_r_new_epoch_3.h5')
print(encoder_model.summary)

<bound method Network.summary of <keras.engine.training.Model object at 0xb4fcb70f0>>


/Library/Anaconda3/anaconda3/envs/env1/lib/python3.5/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


#### Encoder :

In [18]:
enc_inputs = autoencoder_model_loaded.get_layer('enc_input').output
# enc_inputs = Input(shape=(None,), name='enc_input')
# enc_emb = autoencoder_model_loaded.get_layer('shared_emb').get_output_at(0)
enc_lstm = autoencoder_model_loaded.get_layer('enc_lstm').output

encoder_model_loaded = Model(enc_inputs, enc_lstm)
encoder_model_loaded.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
enc_input (InputLayer)       (None, 20)                0         
_________________________________________________________________
shared_emb (Embedding)       (None, 20, 32)            1600096   
_________________________________________________________________
enc_lstm (LSTM)              [(None, 32), (None, 32),  8320      
Total params: 1,608,416
Trainable params: 1,608,416
Non-trainable params: 0
_________________________________________________________________


#### Decoder :

In [19]:
latent_dim = 32

In [20]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

dec_inp = autoencoder_model_loaded.get_layer('dec_input').output
dec_emb = autoencoder_model_loaded.get_layer('shared_emb')(dec_inp)
dec_lstm_layer = autoencoder_model_loaded.get_layer('dec_lstm')

dec_lstm,  decoder_state_output_h, decoder_state_output_c = dec_lstm_layer(dec_emb, initial_state=decoder_input_states)

decoder_output_states = [decoder_state_output_h, decoder_state_output_c]

dec_output = autoencoder_model_loaded.get_layer('dec_output')(dec_lstm)

decoder_model_loaded = Model([dec_inp] + decoder_input_states , [dec_output] + decoder_output_states)

In [21]:
decoder_model_loaded.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dec_input (InputLayer)          (None, 20)           0                                            
__________________________________________________________________________________________________
shared_emb (Embedding)          (None, 20, 32)       1600096     dec_input[0][0]                  
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
dec_lstm (

In [22]:
# index_to_word_dict = dict([(i,w) for w,i in word_to_index.items()])

In [23]:
UNKNOWN_TOKEN = "UNK"
PAD_TOKEN = "PAD"
EOS_TOKEN = "EOS"

unknown_token_idx = word_to_index_dict.get(UNKNOWN_TOKEN).get('0')
pad_token_idx = word_to_index_dict.get(PAD_TOKEN).get('0')
eos_token_idx = word_to_index_dict.get(EOS_TOKEN).get('0')

max_sent_len = 20

## Decode sequence new

In [35]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    op, states_value0, states_value1 = encoder_model_loaded.predict(input_seq)
    print(np.array(states_value0).shape, np.array(states_value1).shape)
#     states_value0 = np.reshape(np.array(states_value0), (20, -1))
#     states_value1 = np.reshape(np.array(states_value1), (20, -1))
#     print(states_value0.shape, states_value1.shape)
    
#     states_value0 = states_value0.tolist()
#     states_value1 = states_value1.tolist()
#     print(states_value1)
    states_value = [states_value0, states_value1]
#     states_value = [states_value0, states_value1]
#     states_value = [np.argmax(states_value0), np.argmax(states_value1)]

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, max_sent_len))
#     target_seq = np.zeros(( max_sent_len, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = word_to_index_dict["PAD"].get('0')
    print(target_seq.shape)
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    i=1
    while not stop_condition :
        output_tokens, h, c = decoder_model_loaded.predict(
            [target_seq] + states_value)
        print("output_tokens shape: ", output_tokens.shape)
        print("output tokens  : ",output_tokens)

        # Sample a token
        sampled_token_index = output_tokens[0, -1, 0]
        print("max index : ", sampled_token_index)
        
        print("predicted word : ", index_to_word_dict.get(int(sampled_token_index)).get('0'))
        sampled_word = index_to_word_dict.get(int(sampled_token_index)).get('0')
        decoded_sentence += ' ' + sampled_word
        
        if i == max_sent_len-1 or sampled_token_index == unknown_token_idx or sampled_token_index == eos_token_idx:
            stop_condition = True
            print("stpping: ", len(decoded_sentence))

        target_seq[0, i] = sampled_token_index

        # Update states
        states_value = [h, c]
        print("dec state shapes: ", np.array(h).shape, np.array(c).shape)
        
        i+=1
        
    return decoded_sentence

## Decode sequence td

In [25]:
def decode_sequence_td(input_seq):
    print(input_seq.shape)
    # Encode the input as state vectors.
    op, states_value0, states_value1 = encoder_model_loaded.predict(input_seq)
    
    print(op.shape)
#     print("enc op : ", index_to_word_dict.get(np.argmax(op[0,-1,:])).get('0'))
    print(np.array(states_value0).shape, np.array(states_value1).shape)
#     print(states_value1)
    states_value = [states_value0, states_value1]

    # Generate empty target sequence of length 1.
    target_seq = np.ones((1, max_sent_len))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = word_to_index_dict.get("EOS").get('0')

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
#     while not stop_condition :
    output_tokens, h, c = decoder_model_loaded.predict(
        [target_seq] + states_value)
    print("output_tokens shape: ", output_tokens.shape)
    print("output tokens  : ",output_tokens)
    
    # Sample a token
    for i in range(20):
        sampled_token_index = int(output_tokens[0, i, 0])
        print("predicted word : ", index_to_word_dict.get(sampled_token_index,"none").get('0'))
#     sampled_word = index_to_word_dict[sampled_token_index]
        decoded_sentence += ' ' + index_to_word_dict.get(sampled_token_index,"none").get('0')

    # Update the target sequence (of length 1).
#     target_seq = np.zeros((1, 1, num_decoder_tokens))
#     target_seq[0, 0, sampled_token_index] = 1.

    # Update states
#     states_value = [h, c]

    return decoded_sentence

## Decode data

In [36]:
import numpy as np

In [37]:
test_str = "Did  ?" #you change your hair
# test_str = "Hey there?"
# text_eval ="Did you change your hair?"
eval_data = np.array([word_to_index_dict.get(str(w), word_to_index_dict.get(UNKNOWN_TOKEN).get('0')).get('0') 
              for w in nlp(test_str)])
print([str(w) for w in nlp(test_str)])
print(eval_data.shape)
print(eval_data)
eval_data.resize(max_sent_len)
eval_data = np.reshape(eval_data, (-1, max_sent_len))
# eval_data = np.reshape(eval_data, (max_sent_len, -1))
print(eval_data)
print("eval data shape", eval_data.shape)
op = decode_sequence(eval_data)

print("output is : ", op)

['Did', ' ', '?']
(3,)
[227   7  20]
[[227   7  20   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]]
eval data shape (1, 20)
(1, 32) (1, 32)
(1, 20)
output_tokens shape:  (1, 20, 1)
output tokens  :  [[[27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]
  [27.518469]]]
max index :  27.518469
predicted word :  be
dec state shapes:  (1, 32) (1, 32)
output_tokens shape:  (1, 20, 1)
output tokens  :  [[[27.554394]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]
  [27.559261]]]
max index :  27.559261
predicted word :  be
dec state shapes:  (1, 32) (1, 32)
outpu